In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats

In [ ]:
name = pd.read_csv('data.tsv', sep='\t',na_values=[r'\N'])

In [ ]:
ratings = pd.read_csv('ratings.tsv', sep='\t',na_values=[r'\N'])

In [ ]:
ratings.dtypes

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(ratings.loc[:,"numVotes"],ratings.loc[:,"averageRating"])
plt.title("Average Rating vs Number of Votes")
plt.ylabel("Average Rating")
plt.xlabel("Number of Votes");

In [ ]:
temp = ratings.filter(['averageRating','numVotes'],axis=1)
temp = temp[temp.columns].astype(float)
from scipy.stats import gaussian_kde
from scipy.interpolate import interpn
y = temp['averageRating']
x = temp['numVotes']
data , x_e, y_e = np.histogram2d( x, y, bins = [15,15])
z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False )
idx = z.argsort()
x, y, z = x[idx], y[idx], z[idx]
plt.figure(figsize=(10, 10))
plt.scatter( x, y, c=z, s=50, edgecolor='')
#plt.xticks(range(1, 10000))
plt.title("Average Rating vs Number of Votes density plot")
plt.ylabel("Average Rating")
plt.xlabel("Number of Votes");
plt.savefig('2.png',dpi = 300)
plt.show()


In [ ]:
basics = pd.read_csv('basics.tsv', sep='\t',na_values=[r'\N'])

In [ ]:
#temp['genres'].str.rstrip('Short')
#basics['extra'] = basics['genres'].str[-5:]
basics = basics.dropna(subset=['runtimeMinutes'])

basics[basics['runtimeMinutes'].astype(int)>90]['extra'].value_counts()

In [ ]:
basics

In [ ]:
basics
ratings
basics = pd.merge(ratings, basics, on='tconst', how='left')

In [ ]:
temp = basics.filter(['averageRating','runtimeMinutes'],axis=1)
temp = temp.dropna()
temp = temp.apply(lambda x: pd.to_numeric(x, errors = 'coerce')).dropna()
temp = temp[temp.columns].astype(float)
temp
    

In [ ]:
temp2 = temp.loc[(temp['runtimeMinutes'] >= 1) & (temp['runtimeMinutes'] <= 240)]
plt.figure(figsize=(10, 10))
plt.scatter(temp2.loc[:,"runtimeMinutes"],temp2.loc[:,"averageRating"])
plt.title("Average Rating vs Number of Votes")
plt.ylabel("Average Rating")
plt.xlabel("Runtime Minutes");

In [ ]:
x = temp2['runtimeMinutes']
y = temp2['averageRating']
bins = [30, 30] # number of bins

# histogram the data
hh, locx, locy = np.histogram2d(x, y, bins=bins)

# Sort the points by density, so that the densest points are plotted last
z = np.array([hh[np.argmax(a<=locx[1:]),np.argmax(b<=locy[1:])] for a,b in zip(x,y)])
idx = z.argsort()
x2, y2, z2 = x[idx], y[idx], z[idx]

plt.figure(1,figsize=(8,8)).clf()
s = plt.scatter(x2, y2, c=z2, cmap='jet', marker='.')  
plt.title("Average Rating vs Runtime density plot")
plt.ylabel("Average Rating")
plt.xlabel("Runtime(minutes)");
plt.savefig('1.png',dpi = 300)
plt.show()

In [ ]:
plt.figure(1,figsize=(8,8)).clf()
s = plt.scatter(x2, y2, c=z2, s=50, cmap='jet',
    edgecolor='', label=None, picker=True, zorder=2) 

In [ ]:
plt.hist2d(x, y, (100, 100), cmap=plt.cm.jet)
plt.colorbar()
plt.title("Average Rating vs Runtime density plot")
plt.ylabel("Average Rating")
plt.xlabel("Runtime(minutes)");
plt.savefig('4.png',dpi = 300)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,7))
temp.groupby(['averageRating']).count()['runtimeMinutes'].plot(ax=ax)
plt.title("Average Rating Distribution")
plt.ylabel("Count")
plt.xlabel("Average Rating");
plt.savefig('5.png',dpi = 300)
plt.show()

In [ ]:
temp = basics.loc[(basics['titleType']=='movie') & (basics['numVotes'] > 10)]
temp
temp=temp.dropna(subset=['averageRating', 'startYear'])
temp

In [ ]:
x = temp['startYear']
y = temp['averageRating']
plt.hist2d(x, y, (30, 30), cmap=plt.cm.jet)
plt.colorbar()
plt.title("Average Rating vs Year of Release density plot")
plt.ylabel("Average Rating")
plt.xlabel("Year of Release");
plt.savefig('7.png',dpi = 300)
plt.show()

In [ ]:
box = pd.read_csv('boxoffice.csv')
awards = pd.read_csv('database.csv')

In [ ]:
awards
awards.Award.unique()
temp = awards.loc[(awards['Award']=='Best Motion Picture') | (awards['Award']=='Best Picture')]
temp

In [ ]:
temp = temp.rename(columns={"Name": "title"})

In [ ]:
temp['title'] = temp['title'].str.rstrip(' ')
temp

In [ ]:
oscar = pd.merge(temp, box, how='left', on='title')
oscar

In [ ]:
oscar.Year = oscar.Year.astype(int)
oscar = oscar.loc[oscar['Year']>1970]
oscar

In [ ]:
oscar = oscar.dropna(subset=['lifetime_gross'])
oscar

In [ ]:
temp3 = oscar.groupby(['Year']).mean()
temp3['limetime_gross2'] = temp3['lifetime_gross']/1000000
temp3

In [ ]:
plt.plot(temp3['year'], temp3['limetime_gross2'])
plt.title("Average Gross of Oscar Nominated Movies vs Yar")
plt.ylabel("Average Gross(Non Inflated)")
plt.xlabel("Year");
plt.savefig('8.png',dpi = 300)
plt.show()

In [ ]:
#xl = pd.read_excel('cpi.xlsx', sheet_name=None)
file = 'cpi.xlsx'
xl = pd.ExcelFile(file)
print(xl.sheet_names)
df1 = xl.parse('Sheet1')
df1 = df1.dropna()


In [ ]:
df1 = df1.rename(columns={"Year": "year"})
temp3.year = temp3.year.astype(int)
df1.year = df1.year.astype(int)
#df1.drop(df1.tail(1).index,inplace=True)
df1

In [ ]:
temp4 = pd.merge(temp3, df1, how='left', on='year')
temp4['inflated'] = temp4['limetime_gross2']*(236/temp4['Annual Average'])
temp4

In [ ]:
plt.plot(temp4['year'], temp4['inflated'])
plt.title("Average Gross of Oscar Nominated Movies vs Year")
plt.ylabel("Average Gross(Inflated)")
plt.xlabel("Year");
plt.savefig('9.png',dpi = 300)
plt.show()

In [5]:
awards = pd.read_csv('query.tsv', sep='\t',na_values=[r'\N'])

In [7]:
awards.tail()


,human,humanLabel,awardEditionLabel,awardLabel,awardWork,awardWorkLabel,director,directorLabel,time
1896,http://www.wikidata.org/entity/Q22101772,Robert Amram,44th Academy Awards,Academy Award for Best Live Action Short Film,http://www.wikidata.org/entity/Q7451352,Sentinels of Silence,NaN,NaN,NaN
1897,http://www.wikidata.org/entity/Q15854394,Wilbur T. Blume,28th Academy Awards,"Academy Award for Best Live Action Short Film,...",http://www.wikidata.org/entity/Q7733033,The Face of Lincoln,NaN,NaN,NaN
1898,http://www.wikidata.org/entity/Q22101716,Joan Keller Stern,42nd Academy Awards,Academy Award for Best Live Action Short Film,http://www.wikidata.org/entity/Q7749449,The Magic Machines,NaN,NaN,NaN
1899,http://www.wikidata.org/entity/Q6245206,John Longenecker,43rd Academy Awards,Academy Award for Best Live Action Short Film,http://www.wikidata.org/entity/Q7760352,The Resurrection of Broncho Billy,NaN,NaN,NaN
1900,http://www.wikidata.org/entity/Q22110362,James Hendrie,62nd Academy Awards,Academy Award for Best Live Action Short Film,http://www.wikidata.org/entity/Q8034562,Work Experience,NaN,NaN,NaN
